# Crowd Counting: Count the Number of Customers Appeared in the Business

![Imgur](https://i.imgur.com/r1U9dHD.png)

##**Scenario**

>"We see our customers as invited guests to a party, and we are the hosts. It’s our job every day to make every important aspect of the customer experience a little bit better."   **--- Jeff Bezos, Amazon CEO**



This is also true for all brick-and-motor stores at shopping malls. As a host, one thing you definitely should know is: *How many `guests` are attending your `party`*.

The #(sign for quantity) of customers are curcial. Key factors for a successful business include `store rent`, `store sales`, which are all strongly correlated with `customers quantity`. \



In Practice, keeping track of passenger flow or customer traffic is never an easy thing. At the old times, customers are counted manually. Later, sensors are installed at the entrance, which provide a higher-accuracy solution, but suffers from high cost (Just imagine how many sensors should be installed). \

Not all malls have the luxury of installing sensors, but all malls have video surveillance cameras. With the images captured by real-time cameras, computer vision technology can help counting customers 24/7.




<img src="https://storage.googleapis.com/kaggle-datasets-images/526740/966025/1b806f39569b1157b1aaafe81f59f4b6/dataset-cover.jpg?t=2020-02-24-06-34-25">

In [1]:
#This step is done to connect the content directory to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Task**
Build a neural network to count the number of customers appeared in the image captured so we can understand the potential customers.


---




If you would like to download the dataset, please use below link.

In [2]:
link = 'https://drive.google.com/file/d/1RwuuNLpQWcozMOGw6Fp42m_dIhuqH90y/view?usp=sharing' # The shareable link


### Unzip the data file
<font color="orange"><b>Todo:</b></font>\
Replace the `PATH_TO_ZIP_FILE` with the path to your data file in your Google Drive. We are unzipping the data file to the `/content/crowd-counting` directory, which won't use your Google Drive storage.

In [3]:
!unzip -q "/content/drive/MyDrive/DAT565E/Individual 3/Crowd_counting_dataset.zip" -d /content/crowd-counting # you should use your own link for the zip file

### Load csv files and train-test-split
<font color="orange"><b>Todo:</b></font>\
(1) Read `/content/crowd-counting/labels.csv`: Read csv files as `Pandas DataFrame` into RAM\
(2) Use `Pandas` API, create a new column named `dir`, which stores the image file names. \
(3) Split the whole data into `training dataframe` (0.7) and `testing dataframe` (0.3)\
(4) Split the above `testing dataframe` into `testing dataframe`(0.5) and `validation dataframe` (0.5)\
Hint: Try to think about the relationship between the number above and the "?" parameters below.

In [4]:
import pandas as pd
df=pd.read_csv("/content/crowd-counting/labels.csv")
X="dir"
Y="count"
df[X]="seq_"+df.id.astype(str).str.zfill(6)+".jpg"

In [5]:
from sklearn.model_selection import train_test_split

# Train-test-validation split
train,_=train_test_split(df,test_size=0.3)
validation,test=train_test_split(_,test_size=0.5)
print(train.shape,validation.shape,test.shape)

(1400, 3) (300, 3) (300, 3)


**Quick check for your answer:**\
(1400, 3) (300, 3) (300, 3)




In [6]:
df.head()

,id,count,dir
0,1,35,seq_000001.jpg
1,2,41,seq_000002.jpg
2,3,41,seq_000003.jpg
3,4,44,seq_000004.jpg
4,5,41,seq_000005.jpg


### Configure data generator
<font color="orange"><b>Todo:</b></font>

Now, let's configure **3** generators: for training, validation, test respectively, in the code chunk below.

\
**Note**: \
(1) Simply rescale input to `1./255`.
You don't need to perform augmentations such as rotation, zooming, and so on. The reason is that the testing images is exactly the same as training images.

(2) Here's a very helpful link about how to use the `flow_from_dataframe` API\
https://keras.io/api/preprocessing/image/

In [47]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

target_size = (120, 160)

train_datagen = ImageDataGenerator(rescale=1./255)
train_gen = train_datagen.flow_from_dataframe(train,
                                              directory='/content/crowd-counting/frames/frames',
                                              target_size=target_size,
                                              class_mode='raw',
                                              x_col=X,
                                              y_col=Y,
                                              shuffle=True)
val_datagen = ImageDataGenerator(rescale=1./255)
val_gen = val_datagen.flow_from_dataframe(validation,
                                          directory='/content/crowd-counting/frames/frames',
                                          target_size=target_size,
                                          class_mode='raw',
                                          x_col=X,
                                          y_col=Y)

test_datagen = ImageDataGenerator(rescale=1./255)
test_gen = test_datagen.flow_from_dataframe(test,
                                            directory='/content/crowd-counting/frames/frames',
                                            target_size=target_size,
                                            class_mode='raw',
                                            x_col=X,
                                            y_col=Y)


Found 1400 validated image filenames.
Found 300 validated image filenames.
Found 300 validated image filenames.


**Quick check:**\
The result should be like: \


Found 1400 validated image filenames.\
Found 300 validated image filenames.\
Found 300 validated image filenames.


### Build Model
<font color="orange"><b>Todo:</b></font>\
Build your convolutional neural network sequential model.
First, build a 2-hidden layer model. Then build a CNN model with 2 convolution layers. Your CNN model should outperform the dense layer model in order to get a full grade.

#### Dense Layer Model

In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.callbacks import EarlyStopping

# Pile up layers.
dense_model = Sequential()
dense_model.add(Flatten(input_shape=(120,160,3)))
dense_model.add(Dense(128, activation='relu'))
dense_model.add(Dense(64, activation='relu'))
dense_model.add(Dense(1))

# Compile model
dense_model.compile(optimizer='adam', loss='mse', metrics=['mae'])
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Fit model.
dense_model.fit(train_gen,
                validation_data=val_gen,
                epochs=50,
                callbacks=[early_stopping])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


44/44 ━━━━━━━━━━━━━━━━━━━━ 11s 132ms/step - loss: 513.4719 - mae: 14.3043 - val_loss: 50.1779 - val_mae: 5.6895
Epoch 2/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 7s 118ms/step - loss: 54.5535 - mae: 5.9812 - val_loss: 42.7021 - val_mae: 5.1438
Epoch 3/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 9s 89ms/step - loss: 44.6007 - mae: 5.3340 - val_loss: 43.8463 - val_mae: 5.5300
Epoch 4/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 5s 105ms/step - loss: 34.7275 - mae: 4.6656 - val_loss: 38.6993 - val_mae: 4.8624
Epoch 5/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 4s 74ms/step - loss: 32.7415 - mae: 4.5522 - val_loss: 24.3025 - val_mae: 3.8521
Epoch 6/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 4s 89ms/step - loss: 25.6213 - mae: 4.0172 - val_loss: 46.2823 - val_mae: 5.8020
Epoch 7/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 5s 103ms/step - loss: 27.6861 - mae: 4.2633 - val_loss: 18.6848 - val_mae: 3.6217
Epoch 8/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 4s 74ms/step - loss: 18.4313 - mae: 3.4478 - val_loss: 14.3648 - val_mae: 3.0792
Epoch 9/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 4s 87ms/step -

The following code is provided for you to calculate model mse.

In [32]:
from sklearn.metrics import mean_absolute_error
import numpy as np
y = []
for i in range(len(test_gen)):
    batch = test_gen[i]
    y.extend(batch[1])
predict = dense_model.predict(test_gen)
mean_absolute_error(np.array(y), predict.flatten())

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step


2.8784710121154786

#### CNN Model

You can use any hyperparameters and improve your results here.

In [52]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau


cnn_model = Sequential()
# Pile up layers.

# Conv+Pool (1)
cnn_model.add(Conv2D(30, (3, 3), activation='relu', input_shape=(120, 160, 3), kernel_regularizer=l2(0.01)))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))

#cnn_model.add(Dropout(0.01))

# Conv+Pool (2)
cnn_model.add(Conv2D(40, (3, 3), activation='relu', kernel_regularizer=l2(0.01)))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))

cnn_model.add(Dropout(0.02))

# Conv+Pool (3)
#cnn_model.add(Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.01)))
#cnn_model.add(MaxPooling2D(pool_size=(2, 2)))

#cnn_model.add(Dropout(0.08))

cnn_model.add(Flatten())

cnn_model.add(Dense(40, activation='relu', kernel_regularizer=l2(0.05)))
#cnn_model.add(Dropout(0.06))

cnn_model.add(Dense(1))


# Compile model
cnn_model.compile(optimizer='adam', loss='mse', metrics=['mae', 'mape'])
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
#model_checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

# Fit model.
cnn_model.fit(train_gen,
                validation_data=val_gen,
                epochs=50,
                callbacks=[early_stopping, reduce_lr])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 11s 154ms/step - loss: 253.6031 - mae: 12.1365 - mape: 40.2579 - val_loss: 54.1358 - val_mae: 5.6086 - val_mape: 18.6211 - learning_rate: 0.0010
Epoch 2/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 4s 89ms/step - loss: 56.0553 - mae: 5.9420 - mape: 20.3871 - val_loss: 40.9280 - val_mae: 4.8379 - val_mape: 17.1977 - learning_rate: 0.0010
Epoch 3/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 4s 87ms/step - loss: 40.5429 - mae: 4.9217 - mape: 16.9182 - val_loss: 25.3814 - val_mae: 3.6092 - val_mape: 11.1790 - learning_rate: 0.0010
Epoch 4/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 4s 85ms/step - loss: 21.6046 - mae: 3.2653 - mape: 11.0557 - val_loss: 16.7760 - val_mae: 2.7999 - val_mape: 9.3840 - learning_rate: 0.0010
Epoch 5/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 4s 84ms/step - loss: 14.6173 - mae: 2.5104 - mape: 8.4129 - val_loss: 14.4647 - val_mae: 2.5278 - val_mape: 8.1233 - learning_rate: 0.0010
Epoch 6/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 4s 77ms/step - loss: 14.9838 - mae: 2.5894 - mape: 8.7167 - val_lo

In [53]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
import numpy as np

def calculate_mae_mape(model, data_gen):
    y_true = []
    for i in range(len(data_gen)):
        y_true.extend(list(data_gen[i][1]))
    y_pred = model.predict(data_gen).flatten()
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred) * 100
    return mae, mape
mae_val, mape_val = calculate_mae_mape(cnn_model, val_gen)
print(f"Validation MAE: {mae_val}")
print(f"Validation MAPE: {mape_val}%")
mae_train, mape_train = calculate_mae_mape(cnn_model, train_gen)
print(f"Train MAE: {mae_train}")
print(f"Train MAPE: {mape_train}%")

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 87ms/step
Validation MAE: 1.8521597131093344
Validation MAPE: 6.148969922601193%
44/44 ━━━━━━━━━━━━━━━━━━━━ 4s 81ms/step
Train MAE: 1.143839567048209
Train MAPE: 3.8710151081685025%


### ***Conclusion and Discussion***

#### **Question 1: Please explain which kinds of layer is used in your CNN model. What are the functions of those layers? Please explain and interpret your results.**



I used 2 convolutional layers (with pool layers) and 1 dense layer. The convolutional layers take partial features from the input pictures and generate the feature figures. The pooling layers function as undersampling, which decrease the level of feature figures to reduce the dimension but keep the most important features. The dense layer plays the role of "decisioning" by learning the weights and combine the features to output the final prediction outcomes. The current result is not ideal. I've tried more than a day of those different parameters. Once I want to add "Dropout" or increase the weight of L2, MAPE both increase a lot, and the overfitting problem still exists. Besides, I believe relative lower neurons works better than larger neurons for each layer through all of my tests. The current result shows that the model lacks the generalization, and there is overfitting.  

#### **Question 2: Which type of error measure did you use for the model training? How does this kind of error measure defined?**



I used mae and mape. MAE is the average absolute difference between the prediction and real value. MAPE is the average relative error between the prediction and real value.  

#### **Question 3: What business problem can be addressed by CNN? Can you give an example?**

Self-driving cars maybe can use CNNs for detecting objects, road lanes, sidewalks, and pedestrains on the road.